In [ ]:
from google.colab import drive
drive.mount('/content/drive')
%cd '/content/drive/MyDrive/SNAP'

In [ ]:
import os
from itertools import product
import numpy as np

data_root = './data/'
processed_data_root = os.path.join(data_root, 'processed/')
figures_root = os.path.join(data_root, 'figures/')

os.makedirs(processed_data_root, exist_ok=True)
os.makedirs(figures_root, exist_ok=True)

device = 'cuda'

In [ ]:
from dimensionality.data_utils import DataProcess

## Gather Data

In [ ]:
dataset = 'dicarlo'
image_type = 'original'  # Only matters for Cadena dataset

regionNames = ['V1',
               'V2',
               #  'V4',
               #  'IT'
               ]

activation_pooling = ["None",
                      #   'MaxPool_(1,1)',
                      #   'AvgPool_(1,1)',
                      ]

rand_projections = ["None",
                    #  "8000",
                    # "12000",
                    ]
pretrained = {True: 'pretrained',
              False: 'untrained'
              }


modelNames = ['alexnet', 'resnet18',
              ]


pooling_list = []
for item in product(activation_pooling, rand_projections):
    pooling_list += ["_RandProj_".join(item)]
activation_pooling = pooling_list.copy()


Data = DataProcess(data_root, activation_pooling, regionNames, modelNames, pretrained)
dfs_all = Data.get_dataframe(load=False, save_all_data_pckl=True)

## Process Data

In [ ]:
trained = True
sort_coord = 'final_scores'
threshold = 0.99

region_list = Data.region_list
pooling_list = Data.pooling_list
model_list = Data.model_list

for region in region_list:
    for pooling in pooling_list:
        print(region, pooling)
        processed_data_name = processed_data_root + f'{region}_{pooling}_{pretrained[trained]}.npz'

        all_data_kwargs = dict(sort_coord=sort_coord,
                               trained=trained,
                               region_list=[region],
                               pooling_list=[pooling],
                               model_list=model_list,
                               eff_dim_cutoff=0,
                               threshold=threshold,
                               )
        all_reg_hist, all_processed_data = Data.get_all_data(**all_data_kwargs)
        all_reg_hist = all_reg_hist[region][pooling]
        all_processed_data = all_processed_data[region][pooling]

        np.savez(processed_data_name, 
                 all_reg_hist=all_reg_hist,
                 all_processed_data=all_processed_data, 
                 all_data_kwargs=all_data_kwargs)

In [ ]:
trained = False
sort_coord = 'final_scores'
threshold = 0.99

region_list = Data.region_list
pooling_list = Data.pooling_list
model_list = Data.model_list

for region in region_list:
    for pooling in pooling_list:
        print(region, pooling)
        processed_data_name = processed_data_root + f'{region}_{pooling}_{pretrained[trained]}.npz'

        all_data_kwargs = dict(sort_coord=sort_coord,
                               trained=trained,
                               region_list=[region],
                               pooling_list=[pooling],
                               model_list=model_list,
                               eff_dim_cutoff=0,
                               threshold=threshold,
                               )
        try:
            all_reg_hist, all_processed_data = Data.get_all_data(**all_data_kwargs)
            all_reg_hist = all_reg_hist[region][pooling]
            all_processed_data = all_processed_data[region][pooling]
        except Exception as e:
            print(e)

        np.savez(processed_data_name, 
                 all_reg_hist=all_reg_hist,
                 all_processed_data=all_processed_data, 
                 all_data_kwargs=all_data_kwargs)